Name: Robert (Bobby) Fisher
CWID: 11187333

In [1]:
# import required modules
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.sql.functions import col, avg
from pyspark.ml.clustering import KMeans

In [2]:
# load iris data into a DataFrame
iris_df = spark.read.load('hdfs://localhost:9000/user/pistolpete/spark_data/iris.data', format="csv")


In [3]:
# examine schema and some rows of data
iris_df.printSchema()
iris_df.show()


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)

+---+---+---+---+------+
|_c0|_c1|_c2|_c3|   _c4|
+---+---+---+---+------+
|5.1|3.5|1.4|0.2|setosa|
|4.9|3.0|1.4|0.2|setosa|
|4.7|3.2|1.3|0.2|setosa|
|4.6|3.1|1.5|0.2|setosa|
|5.0|3.6|1.4|0.2|setosa|
|5.4|3.9|1.7|0.4|setosa|
|4.6|3.4|1.4|0.3|setosa|
|5.0|3.4|1.5|0.2|setosa|
|4.4|2.9|1.4|0.2|setosa|
|4.9|3.1|1.5|0.1|setosa|
|5.4|3.7|1.5|0.2|setosa|
|4.8|3.4|1.6|0.2|setosa|
|4.8|3.0|1.4|0.1|setosa|
|4.3|3.0|1.1|0.1|setosa|
|5.8|4.0|1.2|0.2|setosa|
|5.7|4.4|1.5|0.4|setosa|
|5.4|3.9|1.3|0.4|setosa|
|5.1|3.5|1.4|0.3|setosa|
|5.7|3.8|1.7|0.3|setosa|
|5.1|3.8|1.5|0.3|setosa|
+---+---+---+---+------+
only showing top 20 rows



In [4]:
# using Spark SQL cast some columns to floating point type
iris_df.createOrReplaceTempView("iris")
df_1 = spark.sql("SELECT " + \
	"cast(_c0 AS double) AS slen, " + \
	"cast(_c1 AS double) AS swid, " + \
	"cast(_c2 AS double) AS plen, " + \
	"cast(_c3 AS double) AS pwid, " + \
	"_c4 AS spcs" + \
	" FROM iris")

# examine new schema and some rows of data	
df_1.printSchema()
df_1.show()

root
 |-- slen: double (nullable = true)
 |-- swid: double (nullable = true)
 |-- plen: double (nullable = true)
 |-- pwid: double (nullable = true)
 |-- spcs: string (nullable = true)

+----+----+----+----+------+
|slen|swid|plen|pwid|  spcs|
+----+----+----+----+------+
| 5.1| 3.5| 1.4| 0.2|setosa|
| 4.9| 3.0| 1.4| 0.2|setosa|
| 4.7| 3.2| 1.3| 0.2|setosa|
| 4.6| 3.1| 1.5| 0.2|setosa|
| 5.0| 3.6| 1.4| 0.2|setosa|
| 5.4| 3.9| 1.7| 0.4|setosa|
| 4.6| 3.4| 1.4| 0.3|setosa|
| 5.0| 3.4| 1.5| 0.2|setosa|
| 4.4| 2.9| 1.4| 0.2|setosa|
| 4.9| 3.1| 1.5| 0.1|setosa|
| 5.4| 3.7| 1.5| 0.2|setosa|
| 4.8| 3.4| 1.6| 0.2|setosa|
| 4.8| 3.0| 1.4| 0.1|setosa|
| 4.3| 3.0| 1.1| 0.1|setosa|
| 5.8| 4.0| 1.2| 0.2|setosa|
| 5.7| 4.4| 1.5| 0.4|setosa|
| 5.4| 3.9| 1.3| 0.4|setosa|
| 5.1| 3.5| 1.4| 0.3|setosa|
| 5.7| 3.8| 1.7| 0.3|setosa|
| 5.1| 3.8| 1.5| 0.3|setosa|
+----+----+----+----+------+
only showing top 20 rows



In [5]:
# Compute and print summary statistics
summary_stats = df_1.describe()
summary_stats.show()

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|              slen|               swid|              plen|              pwid|     spcs|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|     null|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   setosa|
|    max|               7.9|                4.4|               6.9|               2.5|virginica|
+-------+------------------+-------------------+------------------+------------------+---------+



In [6]:
# Use feature transformer to produce new column from numeric columns
vec_assembler_1 = VectorAssembler(
    inputCols = ['slen', 'swid', 'plen', 'pwid'],
    outputCol = "numcols")

df_1v = vec_assembler_1.transform(df_1)
df_1v.printSchema()

# Compute Pearson and Spearman correlations
corr_pearson = Correlation.corr(df_1v, "numcols")
corr_spearman = Correlation.corr(df_1v, "numcols", "spearman")

pearson_mat = corr_pearson.take(1)
spearman_mat = corr_spearman.take(1)

print(str(pearson_mat))
print(str(spearman_mat))

root
 |-- slen: double (nullable = true)
 |-- swid: double (nullable = true)
 |-- plen: double (nullable = true)
 |-- pwid: double (nullable = true)
 |-- spcs: string (nullable = true)
 |-- numcols: vector (nullable = true)

[Row(pearson(numcols)=DenseMatrix(4, 4, [1.0, -0.1094, 0.8718, 0.818, -0.1094, 1.0, -0.4205, -0.3565, 0.8718, -0.4205, 1.0, 0.9628, 0.818, -0.3565, 0.9628, 1.0], False))]
[Row(spearman(numcols)=DenseMatrix(4, 4, [1.0, -0.1595, 0.8814, 0.8344, -0.1595, 1.0, -0.3034, -0.2775, 0.8814, -0.3034, 1.0, 0.936, 0.8344, -0.2775, 0.936, 1.0], False))]


In [7]:
# Use feature transformer to prepare DataFrame for 
# input to regression algorithms
vec_assembler_2 = VectorAssembler(
    inputCols = ['swid', 'plen', 'pwid'],
    outputCol = "predictors")

df_2 = vec_assembler_2.transform(df_1)
df_2.printSchema()

# Split DataFrame into training and testing parts
df_train, df_test = df_2.randomSplit([0.6, 0.4], seed = 1234)
print('Train: ', df_train.count())
print('Test : ', df_test.count())

root
 |-- slen: double (nullable = true)
 |-- swid: double (nullable = true)
 |-- plen: double (nullable = true)
 |-- pwid: double (nullable = true)
 |-- spcs: string (nullable = true)
 |-- predictors: vector (nullable = true)

Train:  99
Test :  51


In [8]:
#create linear regression object
lr_obj = LinearRegression(
    maxIter = 200, 
    fitIntercept = True, 
    featuresCol = "predictors",
    labelCol = "slen")

#create random forest object
rf_obj = RandomForestRegressor(
    featuresCol = "predictors",
    labelCol = "slen",
    numTrees = 50, maxDepth = 10)

#create gradient boosted tree object
gbt_obj = GBTRegressor(
    featuresCol = "predictors",
    labelCol = "slen",
    maxIter = 10, maxDepth = 5)

# train the models on the training data
lr_model = lr_obj.fit(df_train)
rf_model = rf_obj.fit(df_train)
gbt_model = gbt_obj.fit(df_train)

In [9]:
# function to compute MSE
def MSE(df, orig_col = "slen", pred_col = "prediction"):
    x = df  \
		.select(((df[pred_col] - df[orig_col]) ** 2).alias("se")) \
        .agg(avg("se"))
    y = x.first()
    return y[0]

# score test data using models
pred_lr = lr_model.transform(df_test)
pred_rf = rf_model.transform(df_test)
pred_gbt = gbt_model.transform(df_test)

# compute and print MSEs
lr_mse = MSE(pred_lr)
rf_mse = MSE(pred_rf)
gbt_mse = MSE(pred_gbt)

print('MSE LR: ', lr_mse)
print('MSE RF: ', rf_mse)
print('MSE GBT: ', gbt_mse)

MSE LR:  0.09908773793596025
MSE RF:  0.10636150308889669
MSE GBT:  0.13808980806776716
